In [1]:
# 0:clicks 1:carts 2:orders
type_weight = {0:1, 1:10, 2:3}
type_weight_multipliers = type_weight

top = 100

VER = 1
DIR = f'/home/anhphantq/vdb/covisit_ver{VER}'

In [2]:
import os 

try:
    os.mkdir(DIR)
    
    os.mkdir(DIR + '/splitted')
    os.mkdir(DIR + '/lb')
except OSError as error:
    print(error)    

[Errno 17] File exists: '/home/anhphantq/vdb/covisit_ver1'


In [3]:
import pandas as pd

# COMPUTE IN PARTS FOR MEMORY MANGEMENT

train_df = pd.read_parquet('/home/anhphantq/otto/splitted_data/train.parquet')
test_df = pd.read_parquet('/home/anhphantq/otto/splitted_data/test.parquet')
all_df = pd.concat((train_df, test_df))
MAX_TS = train_df.ts.max()
MIN_TS = train_df.ts.min()
MAX_TS, MIN_TS

(1661119199, 1659304800)

In [3]:
import pandas as pd
train_df = pd.read_parquet('/home/anhphantq/otto/data/train.parquet')

#remove 1 week from train df LB to retain distribution 
import numpy as np
min_ts = train_df.ts.values.min()
old_index = np.argwhere(train_df.ts.values - min_ts < 3600 * 24 * 7)
train_df = train_df.drop(old_index.flatten())
test_df = pd.read_parquet('/home/anhphantq/otto/data/test.parquet')
all_df = pd.concat((train_df, test_df))
MAX_TS = train_df.ts.max()
MIN_TS = train_df.ts.min()
MAX_TS, MIN_TS

(1661723999, 1659909600)

In [4]:
def create_covisit(df):    
    df = df.sort_values(['session','ts'],ascending=[True,False])

    # USE TAIL OF SESSION
    df = df.reset_index(drop=True)
    df['n'] = df.groupby('session').cumcount()
    df = df.loc[df.n<30].drop('n',axis=1)

    # CREATE PAIRS
    df = df.merge(df,on='session')
    df = df.loc[ ((df.ts_x - df.ts_y).abs()< 1 * 60 * 60) & (df.aid_x != df.aid_y) ]


    # ASSIGN WEIGHTS
    df = df[['session', 'aid_x', 'aid_y','type_y']].drop_duplicates(['session', 'aid_x', 'aid_y', 'type_y'])
    df['wgt'] = df.type_y.map(type_weight)
    df = df[['aid_x','aid_y','wgt']]
    df.wgt = df.wgt.astype('float32')
    
    
    df = df.groupby(['aid_x','aid_y']).wgt.sum().reset_index().sort_values(['aid_x','wgt'],ascending=[True,False]).reset_index(drop = True)
    df['n'] = df.groupby('aid_x').aid_y.cumcount()
    #SAVE TOP 
    df = df.loc[df.n < top].drop('n', axis = 1)
    return df


In [5]:
from tqdm import tqdm
import numpy as np

step = all_df.shape[0] // 64
start = [i * step for i in range(64)]
end = [i + step for i in start]
end[-1] = all_df.shape[0] 

batches = [all_df.iloc[start[i]: end[i]].copy() for i in range(len(start))]




In [6]:
%%time
#splitted

import multiprocessing
with multiprocessing.Pool(64) as pool:
  pd.concat(pool.map(create_covisit, batches, chunksize = 1)).to_parquet(DIR + f'/splitted/covisit')
  
  


CPU times: user 41.8 s, sys: 41.3 s, total: 1min 23s
Wall time: 1min 36s


In [6]:
%%time
#splitted

import multiprocessing
with multiprocessing.Pool(64) as pool:
  pd.concat(pool.map(create_covisit, batches, chunksize = 1)).to_parquet(DIR + f'/lb/covisit')
  
  


create top clicks


100%|██████████| 64/64 [01:28<00:00,  1.38s/it]


create top buy2buy


100%|██████████| 64/64 [00:15<00:00,  4.07it/s]


create top clicks


100%|██████████| 64/64 [00:15<00:00,  4.04it/s]


RECENT covisit

In [5]:
import warnings
from tqdm import tqdm
warnings.filterwarnings("ignore")

In [12]:
from tqdm import tqdm
def create_recent_7_days(df):
    df['date'] = pd.to_datetime(df.ts * 1e9)#7 recent days
    df['dayofyear'] = df.date.dt.dayofyear

    dayofyear_max = df['dayofyear'].max()
    dayofyear_min = dayofyear_max - 6

    df = df.loc[(df.dayofyear >= dayofyear_min) & (df.dayofyear <= dayofyear_max)]
    result = []
    for i in range(dayofyear_min, dayofyear_max + 1):
        result.append(create_covisit(df.loc[df.dayofyear == i]))

    return result
        
   
def pqt_to_dict(df, i, f):
    df['covistation_list'] = list(zip(df.aid_y, df.wgt))
    
    df = df[['aid_x', 'covistation_list']].groupby('aid_x').agg({'covistation_list': list})
    df.to_parquet(f'{f}/covisit{i}.pqt', index = True)

    


    

In [7]:
train_df = train_df.iloc[:1000000]

In [8]:
from tqdm import tqdm
import numpy as np

step = train_df.shape[0] // 64
start = [i * step for i in range(64)]
end = [i + step for i in start]
end[-1] = train_df.shape[0] 

batches = [train_df.iloc[start[i]: end[i]].copy() for i in range(len(start))]




In [13]:
%%time
#splitted

import multiprocessing

with multiprocessing.Pool(64) as pool:
  result = pool.map(create_recent_7_days, batches, chunksize = 1)
for i in tqdm(range(7)):  
    pqt_to_dict(pd.concat([r[i] for r in result]), i, DIR + '/splitted')





 14%|█▍        | 1/7 [00:00<00:03,  1.78it/s]

                                          covistation_list
aid_x                                                     
38         [(292955, 1.0), (1250198, 1.0), (1637451, 1.0)]
114      [(409367, 1.0), (770509, 1.0), (951559, 1.0), ...
295                                       [(1667805, 1.0)]
473      [(620069, 1.0), (1187148, 1.0), (1205968, 1.0)...
579      [(726027, 11.0), (121743, 1.0), (1008578, 1.0)...
...                                                    ...
1854939                    [(189978, 11.0), (292842, 1.0)]
1855198  [(546864, 14.0), (1435478, 14.0), (216329, 1.0...
1855215  [(530268, 1.0), (954262, 1.0), (1004034, 1.0),...
1855464  [(178785, 1.0), (429488, 1.0), (493376, 1.0), ...
1855500  [(474131, 11.0), (1786822, 11.0), (39846, 1.0)...

[15875 rows x 1 columns]


 29%|██▊       | 2/7 [00:01<00:02,  1.82it/s]

                                          covistation_list
aid_x                                                     
3        [(1180285, 11.0), (42746, 1.0), (126949, 1.0),...
4        [(192380, 1.0), (222290, 1.0), (254003, 1.0), ...
259      [(75248, 1.0), (192259, 1.0), (242835, 1.0), (...
605       [(1011435, 1.0), (1562832, 1.0), (1746524, 1.0)]
819      [(225245, 1.0), (287967, 1.0), (694931, 1.0), ...
...                                                    ...
1855357  [(1666159, 11.0), (4498, 1.0), (109883, 1.0), ...
1855464  [(1151818, 11.0), (1596063, 11.0), (39531, 1.0...
1855500  [(812191, 14.0), (648570, 1.0), (738987, 1.0),...
1855518  [(1004467, 1.0), (1023551, 1.0), (1766089, 1.0...
1855576                   [(1410886, 1.0), (1766403, 1.0)]

[15753 rows x 1 columns]


 43%|████▎     | 3/7 [00:01<00:02,  1.81it/s]

...

                                          covistation_list
aid_x                                                     
137                                        [(195833, 1.0)]
183      [(189237, 1.0), (385883, 1.0), (482769, 1.0), ...
220      [(614193, 1.0), (1083921, 1.0), (1436139, 1.0)...
262      [(1026930, 11.0), (210027, 1.0), (274577, 1.0)...
348      [(288980, 1.0), (313004, 1.0), (635472, 1.0), ...
...                                                    ...
1854799  [(259626, 11.0), (444739, 11.0), (1545893, 11....
1854872                     [(312373, 1.0), (469985, 1.0)]
1855154  [(92806, 1.0), (575027, 1.0), (757597, 1.0), (...
1855339      [(219470, 1.0), (728774, 1.0), (926134, 1.0)]
1855500                                   [(1590864, 1.0)]

[14000 rows x 1 columns]


 86%|████████▌ | 6/7 [00:03<00:00,  1.95it/s]

                                          covistation_list
aid_x                                                     
137      [(72319, 14.0), (319580, 14.0), (150132, 1.0),...
195      [(165180, 1.0), (577040, 1.0), (899410, 1.0), ...
259      [(167989, 1.0), (242995, 1.0), (280861, 1.0), ...
984      [(1330306, 14.0), (1450279, 11.0), (35177, 1.0...
1089     [(1353746, 11.0), (1468610, 11.0), (1648502, 1...
...                                                    ...
1855293  [(247477, 1.0), (626201, 1.0), (1329892, 1.0),...
1855333  [(113524, 1.0), (524193, 1.0), (544188, 1.0), ...
1855499  [(850476, 11.0), (418830, 1.0), (771229, 1.0),...
1855500  [(673407, 1.0), (1116095, 1.0), (496180, 1.0),...
1855540  [(561468, 1.0), (989119, 1.0), (1015545, 1.0),...

[14828 rows x 1 columns]


100%|██████████| 7/7 [00:03<00:00,  2.00it/s]

                                          covistation_list
aid_x                                                     
348      [(111958, 1.0), (115739, 1.0), (221895, 1.0), ...
391                       [(316716, 11.0), (542152, 11.0)]
649      [(1846395, 14.0), (1354482, 11.0), (1272754, 4...
984      [(1035092, 11.0), (1788570, 11.0), (35177, 1.0...
1089     [(313238, 1.0), (1071013, 1.0), (1314325, 1.0)...
...                                                    ...
1855368  [(901869, 14.0), (1428004, 14.0), (301361, 13....
1855384  [(7907, 1.0), (20405, 1.0), (44700, 1.0), (207...
1855414  [(99061, 1.0), (333060, 1.0), (688753, 1.0), (...
1855500        [(8752, 1.0), (36824, 1.0), (1426624, 1.0)]
1855508  [(6062, 1.0), (138642, 1.0), (214372, 1.0), (2...

[15045 rows x 1 columns]
CPU times: user 3.72 s, sys: 2.83 s, total: 6.55 s
Wall time: 6.82 s


In [15]:
%%time
#lb

import multiprocessing

with multiprocessing.Pool(64) as pool:
  result = pool.map(create_recent_7_days, batches, chunksize = 1)
for i in tqdm(range(7)):  
    pqt_to_dict(pd.concat([r[i] for r in result]), i, DIR + '/lb')





100%|██████████| 7/7 [00:00<00:00, 34.25it/s]

CPU times: user 392 ms, sys: 1.26 s, total: 1.65 s
Wall time: 1.86 s
